In [4]:
# Core data libraries
import pandas as pd
import numpy as np

# Static visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive plots
import plotly.express as px
import plotly.graph_objects as go
import altair as alt

# Dashboard & widgets
import ipywidgets as widgets
from IPython.display import display, Markdown

# Optional: Map visualizations
import folium
import geopandas as gpd

# Plotting themes
sns.set_theme(style="whitegrid", palette="muted")  # Seaborn theme
plt.rcParams["figure.figsize"] = (10, 6)            # Matplotlib default size
alt.themes.enable("default")                        # Altair theme

# Jupyter display tweaks
%matplotlib inline